In [1]:
from sqlalchemy.types import Enum 
from sqlalchemy import Table, Column, Integer, Numeric, String, CheckConstraint
from sqlalchemy import DateTime, ForeignKey, Boolean, Time, Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, backref
from sqlalchemy.orm import sessionmaker 
from sqlalchemy import create_engine
from datetime import datetime, time, date, timedelta
import pandas as pd
import random

In [2]:
#create DB
engine = create_engine('sqlite:///:memory:')

In [3]:
## Create a session
Session = sessionmaker (bind = engine) #Defines a Session class
session = Session ()  #Creates a session object

In [4]:
Base = declarative_base()

In [5]:
class People(Base):
    
    __tablename__ = 'people'
    
    __table_args__ = ({ 'extend_existing': True})
    
    people_id = Column(Integer(), nullable=False, primary_key = True)
    username = Column(String(10), nullable=False, index=True, unique=True)
    password = Column(String(32), nullable=False)
    confirm_pw = Column(String(32), nullable=False)
    email = Column(String(255), nullable=False, index=True, unique=True)
    first_name = Column(String(10), nullable=False, index=True)
    last_name = Column(String(10), nullable=False, index=True)
    DOB = Column(DateTime, nullable=False, index=True)
    SSN = Column(String(10), nullable=False, index=True, unique=True)
    phone_no = Column(String(20), nullable=False, index=True, unique=True)
    created_on = Column(DateTime, default=datetime.now)
    age = Column(Integer())
    
    def __init__(self, username, password, confirm_pw, email
                , first_name, last_name, DOB, SSN, phone_no, age):
        self.username = username
        self.password = password
        self.confirm_pw = confirm_pw
        self.email = email
        self.first_name = first_name
        self.last_name = last_name
        self.DOB = DOB
        self.SSN = SSN
        self.phone_no = phone_no
        self.age = age
        

    def ConvertToTuples(self):
        tuples = (self.people_id, self.username, self.password, self.confirm_pw, self.email
                , self.first_name, self.last_name, self.DOB, self.SSN, self.phone_no, self.created_on,self.age)
        return tuples    
        

In [6]:
class AddressType(Base):
    __tablename__ = 'addressType'
    
    __table_args__ = ({'extend_existing': True})
    
    addressType_id = Column(Integer(), nullable=False, primary_key= True)
    address_type = Column(Enum('patient', 'personnell', 'site location'), default= 'patient')
    
    def __init__(self, address_type):
        self.address_type = address_type
        
    def ConvertToTuples(self):
        tuples = (self.addressType_id, self.address_type)
        return tuples  

In [7]:
class Address(Base):
    __tablename__ = 'address'
    
    __table_args__ = ({'extend_existing': True})
    
    address_id = Column(Integer(), nullable=False, primary_key=True)
    address_1 = Column(String(32), nullable=False, index=True)
    address_2 = Column(String(32))
    city = Column(String(15), nullable=False)
    state = Column(String(3), nullable=False)
    zip_code = Column(String(6), nullable=False)
    country = Column(String(20), nullable=False)
    addressType_id = Column(Integer(), ForeignKey('addressType.addressType_id'))
    
    addresstype = relationship('AddressType', backref=backref('address', order_by=address_id))
    
    def __init__(self, address_1, address_2, city, state
            , zip_code, country):
        self.address_1 = address_1
        self.address_2 = address_2
        self.city = city
        self.state = state
        self.zip_code = zip_code
        self.country = country
        self.addressType_id = 2
        
    def ConvertToTuples(self):
        tuples = (self.address_id, self.address_1, self.address_2, self.city, self.state
                , self.zip_code, self.country, self.addressType_id)
        return tuples  
    

In [8]:
class SiteLocation(Base):
    __tablename__ = 'siteLocation'
    
    __table_args__ = ({'extend_existing': True})
    
    siteLocation_id = Column(Integer(), nullable= False, primary_key=True)
    site_name = Column(String(255), nullable= False)
    address_id = Column(Integer(), ForeignKey('address.address_id'))
    
    address = relationship('Address', backref=backref('siteLocation', order_by= siteLocation_id))
    
#     def __init__(self, site_name, address_id):
#         self.site_name = site_name
#         self.address_id = address_id
        
    def ConvertToTuples(self):
        tuples = (self.siteLocation_id, self.site_name, self.address_id)
        return tuples 

In [9]:
class PersonalInfo(Base):
    __tablename__ = 'personalInfo'
    
    __table_args__ = ({'extend_existing': True})
    
    personalInfo_id = Column(Integer(), nullable= False, primary_key=True)
    people_id = Column(Integer(), ForeignKey('people.people_id'))
    address_id = Column(Integer(), ForeignKey('address.address_id'))
    
    people = relationship('People', backref=backref('personalInfo', order_by= personalInfo_id))
    address = relationship('Address', backref=backref('personalInfo', order_by= personalInfo_id))
    
    def __init__(self, people_id, address_id):
        self.people_id = people_id
        self.address_id = address_id
        
    def ConvertToTuples(self):
        tuples = (self.personalInfo_id, self.people_id, self.address_id)
        return tuples 

In [10]:
class PersonnellInfo(Base):
    __tablename__ = 'personnellInfo'
    __table_args__ = ({'extend_existing': True})
    
    personnellInfo_id = Column(Integer(), nullable= False, primary_key=True)
    job_title = Column(String(32), nullable= False, index = True)
    people_id = Column(Integer(), ForeignKey('people.people_id'))
    address_id = Column(Integer(),ForeignKey('address.address_id'))
    workAddress_id = Column(Integer(), ForeignKey('siteLocation.siteLocation_id'))
    
    people = relationship('People', backref=backref('personnellInfo', order_by= personnellInfo_id))
    address = relationship('Address', backref=backref('personnellInfo', order_by= personnellInfo_id))
    siteLocation = relationship('SiteLocation', backref=backref('personnellInfo', order_by= personnellInfo_id))
    
    
    def __init__(self, job_title, people_id, address_id, workAddress_id):
        self.job_title = job_title
        self.people_id = people_id
        self.address_id = address_id
        self.workAddress_id = workAddress_id

        
    def ConvertToTuples(self):
        tuples = (self.personnellInfo_id, self.job_title, self.people_id, self.address_id,self.workAddress_id)
        return tuples 

In [11]:
class VaccineAdmin(Base):
    __tablename__ = 'vaccineAdmin'
    
    __table_args__ = ({'extend_existing': True})
    
    vaccineAdmin_id = Column(Integer(), nullable= False, primary_key=True)
    personnellInfo_id = Column(Integer(), ForeignKey('personnellInfo.personnellInfo_id'))
    
    personell = relationship('PersonnellInfo', backref=backref('vaccineAdmin', order_by=vaccineAdmin_id))
    
    def __init__(self, personnellInfo_id):
        self.personnellInfo_id = personnellInfo_id
        
    def ConvertToTuples(self):
        tuples = (self.vaccineAdmin_id, self.personnellInfo_id)
        return tuples 

In [12]:
class Vaccine(Base):
    __tablename__ = 'vaccine'
    __table_args__ = ({'extend_existing': True})
    
    vaccine_id = Column(Integer(), nullable= False, primary_key=True)
    name = Column(Enum('BNT162b2', 'mRNA1273', 'JNJ78436735'), nullable= False, index=True)
    manufacturer = Column(Enum('PfizerBioNTech', 'ModernaTXInc', 'JohnsonAndJohnson'), nullable= False, index = True) 
    ailments = Column(String(25))
    dose = Column(Integer(), nullable= False, index=True)
    days_apart = Column(Integer())
    vaccine_type = Column(String(32))
    comments = Column(String(255))
       
    
    def __init__(self, name, manufacturer, ailments, dose, days_apart, vaccine_type, comments):
        self.name = name
        self.manufacturer = manufacturer
        self.ailments = ailments
        self.dose = dose
        self.days_apart = days_apart
        self.vaccine_type = vaccine_type
        self.comments = comments

        
    def ConvertToTuples(self):
        tuples = (self.vaccine_id, self.name, self.manufacturer, self.ailments, self.dose, self.days_apart,self.vaccine_type, self.comments)
        return tuples 

In [13]:
class Appointment(Base):
    __tablename__ = 'appointment'
    __table_args__ = ({'extend_existing': True})
    
    appointment_id = Column(Integer(), nullable= False, primary_key=True)
    personalInfo_id = Column(Integer(), ForeignKey('personalInfo.personalInfo_id'))
    vaccineAdmin_id = Column(Integer(), ForeignKey('vaccineAdmin.vaccineAdmin_id'))
    appt_date= Column(Date)
    appt_time = Column(Time)
    #alter table to create a column for second appt two weeks after first appt
    sec_appt_date = Column(Date)
    sec_appt_time = Column(Time)
    location_id = Column(Integer(), ForeignKey('siteLocation.siteLocation_id'))
    vaccine_id = Column(Integer(), ForeignKey('vaccine.vaccine_id'))
    comments = Column(String(255))
    
    
    personalInfo = relationship('PersonalInfo', backref=backref('appointment', order_by= appointment_id))
    vaccineAdmin = relationship('VaccineAdmin', backref=backref('appointment', order_by= appointment_id))
    siteLocation = relationship('SiteLocation', backref=backref('appointment', order_by= appointment_id))
    vaccine = relationship('Vaccine', backref=backref('appointment', order_by= appointment_id))
    
    
    
    def __init__(self,personalInfo_id, vaccineAdmin_id, appt_date, appt_time, location_id, vaccine_id, comments):
        self.personalInfo_id = personalInfo_id
        self.vaccineAdmin_id = vaccineAdmin_id
        self.appt_date = appt_date
        self.appt_time = appt_time
        self.location_id = location_id
        self.vaccine_id = vaccine_id
        self.comments = comments

        
    def ConvertToTuples(self):
        tuples = (self.appointment_id, self.personalInfo_id
                  , self.vaccineAdmin_id, self.appt_date,
                  self.appt_time, self.sec_appt_date
                  ,self.sec_appt_time, self.location_id
                  , self.vaccine_id, self.comments)
        return tuples 

In [14]:
class Patient(Base):
    __tablename__ = 'patient'
    
    __table_args__ = ({'extend_existing': True})
    
    patient_id = Column(Integer(), nullable= False, primary_key=True)
    personalInfo_id = Column(Integer(), ForeignKey('personalInfo.personalInfo_id'))
    appointment_id = Column(Integer(), ForeignKey('appointment.appointment_id'))
    
    
    personalInfo = relationship('PersonalInfo', backref=backref('patient', order_by=patient_id))
    appointment = relationship('Appointment', backref=backref('patient', order_by=patient_id))

    
    def __init__(self, personalInfo_id,appointment_id):
        self.personalInfo_id = personalInfo_id
        self.appointment_id = appointment_id
        
    def ConvertToTuples(self):
        tuples = (self.patient_id, self.personalInfo_id,self.appointment_id)
        return tuples 

In [15]:
class Sessions(Base):
    __tablename__ = 'session'
    
    __table_args__ = ({'extend_existing': True})
    
    session_id = Column(Integer(), nullable= False, primary_key=True)
    patient_id = Column(Integer(), ForeignKey('patient.patient_id'))
    appointment_id = Column(Integer(), ForeignKey('appointment.appointment_id'))
    vaccine_id = Column(Integer(), ForeignKey('vaccine.vaccine_id'))
    vaccineAdmin_id = Column(Integer(), ForeignKey('vaccineAdmin.vaccineAdmin_id'))
     #alter table to create column
    dose_rem = Column(Integer())
    
    patient = relationship('Patient', backref=backref('session', order_by=session_id))
    appointment = relationship('Appointment', backref=backref('session', order_by=session_id))
    vaccine = relationship('Vaccine', backref=backref('session', order_by=session_id))
    vaccineAdmin = relationship('VaccineAdmin', backref=backref('session', order_by=session_id))
    
    
    def __init__(self, patient_id,appointment_id, vaccine_id, vaccineAdmin_id):
        self.patient_id = patient_id
        self.appointment_id = appointment_id
        self.vaccine_id = vaccine_id
        self.vaccineAdmin_id = vaccineAdmin_id
        
    def ConvertToTuples(self):
        tuples = (self.session_id, self.patient_id,self.appointment_id
                  , self.vaccine_id, self.vaccineAdmin_id, self.dose_rem)
        return tuples 

In [16]:
Base.metadata.create_all(engine)   

In [17]:
# ll = {'df1':'people','df2':'address_type','df3':'address', 'df4':'siteLocation','df5':'personalInfo'
#      ,'df6':'personnellInfo','df7':'vaccineAdmin', 'df8':'vaccine', 'df9':'appointment','df10':'patient','df11':'session'}
   

In [18]:

# for key in ll:
#     with pd.ExcelFile("data.xlsx") as xls:
#         key = pd.read_excel(xls, ll[key])
#         key.to_sql(ll[key], con = engine, if_exists= 'append',index_label='people_id', index=False)

# ll['created_on'] = pd.to_datetime("now")

    

In [19]:
# Reads data from multiple sheets in our excel files
with pd.ExcelFile("data.xlsx") as xls:
    df1 = pd.read_excel(xls, "people")
    df2 = pd.read_excel(xls, "address_type")
    df3 = pd.read_excel(xls, "address")
    df4 = pd.read_excel(xls, "siteLocation")
    df5 = pd.read_excel(xls, "personalInfo")
    df6 = pd.read_excel(xls, "personnellInfo")
    df7 = pd.read_excel(xls, "vaccineAdmin")
    df8 = pd.read_excel(xls, "vaccine")
    df9 = pd.read_excel(xls, "appointment")
    df10 = pd.read_excel(xls, "patient")
    df11 = pd.read_excel(xls, "session")

    
# adds the created_on column and gives it a default date
df1['created_on'] = pd.to_datetime("now")


In [20]:
# add the data from each file and store in DB
df1.to_sql('people', con = engine, if_exists= 'append',index_label='people_id', index=False)

df2.to_sql('address_type', con = engine, if_exists= 'append',index_label='addressType_id', index=False)

df3.to_sql('address', con = engine, if_exists= 'append',index_label='address_id', index=False)

df4.to_sql('siteLocation', con = engine, if_exists= 'append',index_label='siteLocation_id', index=False)

df5.to_sql('personalInfo', con = engine, if_exists= 'append',index_label='personalInfo_id', index=False)

df6.to_sql('personnellInfo', con = engine, if_exists= 'append',index_label='personnellInfo_id', index=False)

df7.to_sql('vaccineAdmin', con = engine, if_exists= 'append',index_label='vaccineAdmin_id', index=False)

df8.to_sql('vaccine', con = engine, if_exists= 'append',index_label='vaccine_id', index=False)

df9.to_sql('appointment', con = engine, if_exists= 'append',index_label='appointment_id', index=False)

df10.to_sql('patient', con = engine, if_exists= 'append',index_label='patient_id', index=False)

df11.to_sql('session', con = engine, if_exists= 'append',index_label='session_id', index=False)




In [21]:
# Obtain a Cursor object to execute SQL statements in SQLITE because
# SQLALCHEMY does not have provisions for altering/updating a table or column in SQLITE
cur = engine.connect()


In [22]:
# confirming receipt of user's info using pandas
def show_dataframe(relation):
    q = session.query(relation)

    ls = []
    print("Showing {} information".format(relation.__table__))
    for obj in q:
        # put tuples in a list to get a list of tuples
        tp = obj.ConvertToTuples()
        ls.append(tp)

        col = relation.__table__.columns.keys()
        ids = col[0]
    # Creating a dataframe that can be used for data analysis in Python
    df = pd.DataFrame(data = ls, columns = col) # Creating a Dataframe out of query result
    df.set_index(ids, inplace=True)
    return df


In [23]:
dtf = show_dataframe(Vaccine)
dtf

Showing vaccine information


,name,manufacturer,ailments,dose,days_apart,vaccine_type,comments
vaccine_id,,,,,,,
1,BNT162b2,PfizerBioNTech,COVID,2,21,mRNA,Shot in the muscle of the upper arm.Does NOT C...
2,mRNA1273,ModernaTXInc,COVID,2,28,mRNA,Shot in the muscle of the upper arm. Does NOT ...
3,JNJ78436735,JohnsonAndJohnson,COVID,1,0,viral vector,Shot in the muscle of the upper arm. Does NOT ...


In [24]:
# Get person data from input

def get_user_data():
    mydict = {}
    param = ['username','password','confirm_pw', 'email'
             , 'first_name', 'last_name', 'DOB (yyyy-mm-dd)', 'SSN'
             , 'phone_no']
    
    print("Fill in your details below: ")
    print(" ")
    for i in param:
        print("what is your {}: ".format(i))
        a = input()
        mydict[i] = a;

    y, m , d = (int(x) for x in mydict['DOB (yyyy-mm-dd)'].split('-'))

    person1= People(
        username = mydict['username'],
        password = mydict['password'],
        confirm_pw = mydict['confirm_pw'],
        email = mydict['email'],
        first_name = mydict['first_name'],
        last_name = mydict['first_name'],
        DOB = datetime(y,m,d),
        SSN = mydict['SSN'],
        phone_no = mydict['phone_no'],
        age = 0
        )
    return person1

In [25]:
# Get user's address
def get_user_address():
    mydict = {}
    param = ['address_1','address_2','city', 'state'
             ,'zip_code', 'country']

    print("Fill in your Address below: ")
    print(" ")
    for i in param:
        print("{}: ".format(i))
        a = input()
        mydict[i] = a
        
    addr1 = Address (
            address_1 = mydict['address_1'],
            address_2 = mydict['address_2'],
            city = mydict['city'],
            state = mydict['state'],
            zip_code = mydict['zip_code'],
            country = mydict['country'],
            )
    return addr1

In [26]:
# Register user by creating a personal info object
# returns user object from DB
def register_user(user, address):
    peopleid = user.people_id
    addressid = address.address_id
    
    user1 = PersonalInfo(peopleid,addressid)

    # relating the people Object to a patient
    user1.people = user

    # Add address info for patient
    user1.address = address
    
    user1.people.created_on = datetime.now()
    # add user to session
    session.add(user1)
    
    # commiting user info to DB
    session.commit()
    return user1.personalInfo_id
    
# register_user()

In [27]:
# creates a column for age on people relation
def create_age_column():
    updateColumn = "UPDATE people SET age = created_on - DOB;"
    cur.execute(updateColumn)
    session.commit()


In [28]:
# User books an appointment
def book_user_appointment(user_id):
    mydict = {}
    params = ['date (yyyy-mm-dd)','time (hh:mm)']
    print("Do you want to book an appoinment (Y or N): ")
    a = input()
    ans = a.upper()
    if ans == 'N':
        print('Keep safe. Don\'nt get COVID')
        return
        
    # Get date and time
    for i in params:
        print("Enter {} : ".format(i))
        a = input()
        mydict[i] = a

    loc = ''
    while(loc != '1' and loc != '2' and loc != '3'):
          # Get Location
        print("Choose a Location\n")
        print("1 - Meharry Medical College, 2 - Saint Thomas , 3 - Vanderbilt Medical \n")
        loc = input()
       
    q = session.query(SiteLocation).all()
    for i in q:
        if loc == '1' and i.siteLocation_id == int(loc):
            mydict['location'] = i.siteLocation_id
        elif loc == '2'and i.siteLocation_id == int(loc):
            mydict['location'] = i.siteLocation_id
        elif loc == '3'and i.siteLocation_id == int(loc):
            mydict['location'] = i.siteLocation_id
 
    vac = ''
    while(vac != '1' and vac != '2' and vac != '3'):
          # Get vaccine type
        print("Choose a vaccine\n")
        print("1 - PfizerBioNTech, 2 - ModernaTXIncs , 3 - JohnsonAndJohnson \n")
        vac = input()
     
    q = session.query(Vaccine).all()
    
    for i in q:
        if vac == '1'and i.vaccine_id == int(loc):
            mydict['vaccine'] = i.vaccine_id
        if vac == '2'and i.vaccine_id == int(loc):
            mydict['vaccine'] = i.vaccine_id
        if vac == '3'and i.vaccine_id == int(loc):
            mydict['vaccine'] = i.vaccine_id
    
    # get any comments
    print("Do you have any comments: ")
    comm = input()
    
    # randomly assign a vaccine administrator
    q = session.query(VaccineAdmin.vaccineAdmin_id).count()
    c = int(q)
    n = random.randint(1,c)
    
    # convert date and time string to datetime objects
    d = mydict['date (yyyy-mm-dd)']
    t = mydict['time (hh:mm)']
    
    
    # create appointment object
    appt = Appointment (
            personalInfo_id = user_id,
            vaccineAdmin_id = n,
            appt_date = datetime.strptime(d, '%Y-%m-%d').date(),
            appt_time = datetime.strptime(t, '%H:%M').time(),
            location_id = mydict['location'],
            vaccine_id = mydict['vaccine'],
            comments = comm
           )
    
    return appt
 

In [29]:
# assign second appointment based on days apart
def update_Appointment():
    q = session.query(Appointment).join(Vaccine).filter(
        Appointment.vaccine_id == Vaccine.vaccine_id)
    
    for i in q:
        day = i.vaccine.days_apart
        appt = i.appt_date
        appt_id = i.appointment_id
        
        futuredate = appt + timedelta(days=day)
        x = (futuredate,appt_id)
        sec_appt_dates = "UPDATE appointment SET sec_appt_date = ? WHERE appointment_id = ?"
        cur.execute(sec_appt_dates,x)
        session.commit()

        # for time
        sec_appt_time = "UPDATE appointment SET sec_appt_time = appt_time;"
        cur.execute(sec_appt_time)
        session.commit()


In [30]:
# Assume the USER made the appointment and took the VACCINE
# UPDATE the DB accordingly, add info on SESSION
# make the VACCINATED USER a PATIENT
def update_patient_record(appt_id):
    q = session.query(Appointment).filter(Appointment.appointment_id == appt_id).one()
    pat = Patient(q.personalInfo_id, q.appointment_id)
    session.add(pat)
    session.commit()
    return pat.patient_id
    
def add_user_to_session(pat_id):
    q = session.query(Patient.personalInfo_id,Appointment.appointment_id
                      , Vaccine.vaccine_id,Vaccine.dose,VaccineAdmin.vaccineAdmin_id)
    q1 = q.join(Vaccine).join(VaccineAdmin)
    r = q1.filter(Patient.patient_id == pat_id).first()
    
    # Make a Sessions Object
    ss = Sessions(r.personalInfo_id, r.appointment_id, r.vaccine_id,r.vaccineAdmin_id)

    session.add(ss)
    session.commit()

In [31]:
 # Count the dose left after patient's session and ALTER
        # table and creat a column to INSERT the value
def update_dose():
    q=session.query(Sessions).join(Vaccine).filter(Sessions.vaccine_id == Vaccine.vaccine_id)
    for i in q:
        dose_rem =  int(i.vaccine.dose) - 1 or 0
        
        pat_id = i.patient_id
        
        if dose_rem < 0:
            dose_rem = 0
    
        sql = "UPDATE session SET dose_rem = ? WHERE patient_id == ?"
        x = (dose_rem,pat_id)
        cur.execute (sql, x)
        session.commit()

In [32]:
# send a message to user
l = []
def send_message(appt_id):
    q = session.query(Appointment.appointment_id,Appointment.appt_date
                      ,Appointment.appt_time,Appointment.personalInfo_id
                      ,SiteLocation.site_name,Vaccine.vaccine_type
                      ,Vaccine.name,Vaccine.manufacturer, Vaccine.comments
                      , Vaccine.dose)
    q = q.join(Vaccine).join(SiteLocation)
    
    r = q.filter(Appointment.appointment_id == appt_id)
    
    print(appt_id)
    for i in r:
        appt_date = i.appt_date
        appt_time = i.appt_time
        site_name = i.site_name
        appointment_id = i.appointment_id
        personalInfo_id = i.personalInfo_id
        name = i.name
        vaccine_type = i.vaccine_type
        manufacturer = i.manufacturer
              
    print('Your appointment is set for {} at {}, at the {}.'.format(appt_date,appt_time,site_name))
    print('Your appointment ID is {}-{}-{}.'.format(appointment_id,vaccine_type,personalInfo_id))
    print('You will be receiving the {}. It is an {} type vaccine manufactured by {}'.format(name
    ,vaccine_type, manufacturer))
    x = i.comments.split('.')[0]
    print('Method of Delivery: {}'.format(x))
  
       

In [33]:
def vaccine_registration_process():
    
    # Get user personal info from input
    user1 = get_user_data()
    
    # Get user address info from input
    address1 = get_user_address()
  
    # Register user
    user1_id = register_user(user1, address1)
    
    session.add(user1)
    session.add(address1)
    session.commit()
    
    # After user has been stored in DB
    # Alter people table to add age column for user
    # calculate age from created_on and store user's age
    create_age_column()
    
    # User books appoinment
    appt = book_user_appointment(user1_id) 
    session.add(appt)
    session.commit()
    
    update_Appointment()
    
    # Assume the USER made the appointment and took the VACCINE
    # UPDATE the DB accordingly
    # make the VACCINATED USER a PATIENT
    # and populate the SESSION table
    pat_id = update_patient_record(appt.appointment_id)
    
    add_user_to_session(pat_id)
    
    update_dose()
 
    # issue message
    send_message(appt.appointment_id)
    

In [34]:
vaccine_registration_process()


Fill in your details below: 
 
what is your username: 
rety
what is your password: 
ewrt
what is your confirm_pw: 
ewr
what is your email: 
qewr
what is your first_name: 
ewr
what is your last_name: 
ewr
what is your DOB (yyyy-mm-dd): 
1988-09-09
what is your SSN: 
sdfg
what is your phone_no: 
rt
Fill in your Address below: 
 
address_1: 
sfdfg
address_2: 
ewrt
city: 
ret
state: 
rt
zip_code: 
ret
country: 
fdfg
Do you want to book an appoinment (Y or N): 
y
Enter date (yyyy-mm-dd) : 
2020-07-09
Enter time (hh:mm) : 
12:30
Choose a Location

1 - Meharry Medical College, 2 - Saint Thomas , 3 - Vanderbilt Medical 

1
Choose a vaccine

1 - PfizerBioNTech, 2 - ModernaTXIncs , 3 - JohnsonAndJohnson 

2
Do you have any comments: 
fdgfwert
4
Your appointment is set for 2020-07-09 at 12:30:00, at the Meharry Medical College.
Your appointment ID is 4-mRNA-4.
You will be receiving the BNT162b2. It is an mRNA type vaccine manufactured by PfizerBioNTech
Method of Delivery: Shot in the muscle of th

In [35]:
dfp = show_dataframe(People)
dfs = show_dataframe(Sessions)
dfa = show_dataframe(Appointment)
dfa

dfa.to_csv(r'C:\Users\markokafor\panda_export.csv', header=True)

Showing people information
Showing session information
Showing appointment information


In [36]:
dfa = show_dataframe(Sessions)
dfa

Showing session information


,patient_id,appointment_id,vaccine_id,vaccineAdmin_id,dose_rem
session_id,,,,,
1,2,1,2,2,1
2,1,3,2,2,1
3,3,2,3,1,0
4,4,1,1,2,1


In [37]:
dfa.to_csv(r'C:\Users\markokafor\panda_export.csv', header=True)